# Self-traind CNN

## Imports

In [1]:
import tensorflow as tf
import os
import shutil
import sys
if(sys.prefix != sys.base_prefix):
    print('Current Conda Environment: {}'.format(os.environ['CONDA_DEFAULT_ENV']))
else:
    print('Errro: Not in virtual environment')
    
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()
if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    

Errro: Not in virtual environment
The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  1 



In [90]:
# import necessary package
from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.initializers import glorot_uniform

import numpy as np
import pandas as pd 
import pathlib
import datetime
import keras

# printout versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"numpy Version: {np.version.version}")
print(f"Keras: {keras.__version__}")

Tensor Flow Version: 2.1.0
numpy Version: 1.18.5
Keras: 2.3.0


### Enum für Training-Set

In [29]:
from enum import Enum

class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Output Directory

* <i>SSD</i>, falls genug Speicher auf SSD im SymLink <i>fast_output</i> verfügbar ist
* <i>HDD</i>, falls möglicherweise zu wenig SSD-Speicher verfügbar ist $\rightarrow$ <i>output</i>

In [30]:
from enum import IntEnum

class OutputDirectory(IntEnum):
    HDD = 0
    SSD = 1
    
output_path = ['output', 'fast_output']

### Convert Label_Type into suitable label names.
$\Rightarrow$ Angular / Normalized $\rightarrow$ ['Elevation', 'Azimuth']

$\Rightarrow$ Stereographic $\rightarrow$ ['S_x', 'S_y']

In [31]:
def get_Label_Names(label_type):
    if label_type == 'Angular' or label_type == 'Normalized' or label_type == 'Proto':
        return ['Elevation', 'Azimuth']
    elif label_type == 'Stereographic':
        return ['S_x', 'S_y']
    else:
        assert(True, 'LabelType Invalid')
        return None

### Benutzerdefinierte Kostenfunktion & Metrik

In [32]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis = -1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis = -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Convert String into Reduction Metric Function

In [33]:
def get_Reduction_Metric(metric):
    
    if metric == 'custom_mae':
        return [custom_mae]
    elif metric == 'tf.keras.metrics.MeanAbsoluteError()':
        return [tf.keras.metrics.MeanAbsoluteError()]
    elif metric == 'circular_mae':
        return [circular_mae]
    elif metric == 'mean_squared_error':
        return ['mean_squared_error']
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Trainingsset-Typ nach String Converter

In [34]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

## AlexNet

In [94]:
#AlexNet with batch normalization in Keras 
def alexnet(activation='relu', input_shape=(224,224,3)):
    #model = Sequential()
    #model.add(Conv2D(96, (11, 11), stride=(4, 4), padding='valid', input_shape=input_shape))
    #model.add(BatchNormalization())
    #model.add(Activation(activation))
    #model.add(MaxPooling2D(pool_size=(3, 3)))
#
    #model.add(Conv2D(256, (5, 5), padding='valid'))
    #model.add(BatchNormalization())
    #model.add(Activation(activation))
    #model.add(MaxPooling2D(pool_size=(3, 3)))
#
    #model.add(Conv2D(384, (3, 3), padding='valid'))
    ##model.add(BatchNormalization())
    ##model.add(Activation(activation))
    ##model.add(MaxPooling2D(pool_size=(3, 3)))
#
    #model.add(Conv2D(384, (3, 3), padding='valid'))
    ##model.add(BatchNormalization())
    ##model.add(Activation(activation))
    ##model.add(MaxPooling2D(pool_size=(3, 3)))
#
    #model.add(Conv2D(256, (3, 3), padding='valid'))
    ##model.add(BatchNormalization())
    ##model.add(Activation(activation))
    ##model.add(MaxPooling2D(pool_size=(3, 3)))
    #
    ##model.add(Flatten())
    ##model.add(Dense(4096))
    ##model.add(BatchNormalization())
    ##model.add(Activation(activation))
    ##model.add(Dense(4096))
    ##model.add(BatchNormalization())
    ##model.add(Activation(activation))
    ##model.add(Dense(1000))
    ##model.add(BatchNormalization())
    ##model.add(Activation('softmax'))
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(units = 2, activation='relu')

        #keras.layers.Dense(10, activation='softmax')
    ])

    return model

In [58]:
def alexnetK2(activation='relu', input_shape=(224,224,3)):
    image_input = Input(input_shape)
    vector_input = Input((12,))
    
    image_model = Conv2D(32,(8,8), strides=(4,4))(image_input)
    image_model = Activation(activation)(image_model)
    image_model = Conv2D(64,(4,4), strides=(2,2))(image_model)
    image_model = Activation(activation)(image_model)
    image_model = Conv2D(64,(3,3), strides=(1,1))(image_model)
    image_model = Activation(activation)(image_model)
    image_model = Flatten()(image_model)
    image_model = Dense(512)(image_model)
    image_model = Activation(activation)(image_model)

    value_model = Dense(16)(vector_input)
    value_model = Activation(activation)(value_model)
    value_model = Dense(16)(value_model)
    value_model = Activation(activation)(value_model)
    value_model = Dense(16)(value_model)
    value_model = Activation(activation)(value_model)

    merged = concatenate([image_model, value_model])

    output = Dense(1, activation='sigmoid')(merged)

    model = Model(inputs=[image_input, vector_input], outputs=output)

    model.compile(loss='binary_crossentropy', optimizer='adam')

## VGG16

In [59]:
def vgg16():
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same')(img_input))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same')(x))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same')(x))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same')(x))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x))
             
    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x))
             
    #Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same')(x))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x))

## VGG6

In [60]:
def vgg6():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides = 1, padding = "same", activation = "relu", input_shape = (32, 32, 3)))
    model.add(Dropout(0.4))
    model.add(Conv2D(32, (3, 3), strides = 1, padding = "same", activation = "relu"))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides = 1, padding = "same", activation = "relu"))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), strides = 1, padding = "same", activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Conv2D(128, (3, 3), strides = 1, padding = "same", activation = "relu"))
    model.add(Dropout(0.4))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    return model

### Parameter

In [61]:
trainingset = TrainingSet.SYNTHETIC
_USE_DATA_AUGMENTATION = True

_BATCH_SIZE = 32 #64
_IMG_HEIGHT = 224
_SAMPLES = 64
_EPOCHS = 1
_OPTIMIZER = Adam
_LR = 1 #2, 5
_FIRST_NEURON = 1024 #2048, 4096
_DROP_OUT = 0.25 # 0.5
_ACTIVATION = 'relu' # leakyrelu
_HIDDEN_LAYERS = 0 #1, 2, 3, 4

_RUN = 'SYNTH'
_LOSS = 'mean_squared_error'
_DATASET_NAME = '201019_2253_final'
_DEVICE = 'GeForce_RTX_2070'
storage = OutputDirectory.SSD

_REDUCTION_METRIC = 'custom_mae'
_MONITOR_VALUE = 'val_custom_mae'
_LABLE_TYPE = 'Proto' # Stereographic, Angular, Normalized
_DEVICE = 'RTX_2070_GPU0'

In [62]:
#storage = OutputDirectory.SSD # 'fast_output' if ssd storage may suffice, 'output' otherwise

#APPENDIX = 'Stereographic'

#FUNCTION_OVERRIDE = ['mean_squared_error', [custom_mae], 'val_custom_mae'] # None, or e. g. ['mean_squared_error', [circular_mae], 'val_circular_mae']

if _LABLE_TYPE == 'Stereographic':
    _CSV_FILE_NAME = 'images_synthetisch_stereographic.csv'
    _IMAGE_TYPE_NAME = ''
    _COLOR_MODE = 'rgb'
    _COLOR_CHANNELS = 3
    _STEREOGRAPHIC = True
elif _LABLE_TYPE == 'Angular':
    _CSV_FILE_NAME = 'labels_ks_rgb.csv'
    _IMAGE_TYPE_NAME = 'rgb'
    _COLOR_MODE = 'rgb'
    _COLOR_CHANNELS = 3
    _STEREOGRAPHIC = False
elif _LABLE_TYPE == 'Angular_RGBD':
    _CSV_FILE_NAME = 'labels_ks_rgbd.csv'
    _IMAGE_TYPE_NAME = 'rgbd'
    _COLOR_MODE = 'rgba'
    _COLOR_CHANNELS = 4
    _STEREOGRAPHIC = False
elif _LABLE_TYPE == 'Proto':
    _CSV_FILE_NAME = 'lables_ks_proto.csv'
    _IMAGE_TYPE_NAME = 'rgb_proto'
    _COLOR_MODE = 'rgb'
    _COLOR_CHANNELS = 3
    _STEREOGRAPHIC = False
elif _LABLE_TYPE == 'Normalized':
    _CSV_FILE_NAME = 'images_synthetisch_normalized.csv'
    _IMAGE_TYPE_NAME = ''
    _COLOR_MODE = 'rgb'
    _COLOR_CHANNELS = 3
    _STEREOGRAPHIC = False
else:
    assert(True, 'LabelType Invalid')

In [63]:
_IMAGE_DIR = '../../data_generation/dataset/{}/{}/'.format(_DATASET_NAME, _IMAGE_TYPE_NAME)
#_IMAGE_DIR = '../../data_generation/'
_CSV_FILE = _IMAGE_DIR + _CSV_FILE_NAME
print(_CSV_FILE)
data_dir = pathlib.Path(_IMAGE_DIR)
_IMAGE_COUNT = len(list(data_dir.glob('*.png')))
_STEPS_PER_EPOCH = np.ceil(_IMAGE_COUNT/_BATCH_SIZE)

_note = '_Custom-MAE'

_MODEL_DIR = '..\\output\\{}_Regression_{}\\{}_{}_Base{}\\'.format(_RUN, _LOSS, _DATASET_NAME, _LABLE_TYPE, _note)
_NET_DIR = '{}_Regression_{}\\{}_{}_Top_1{}\\{}_TD\\'.format(_RUN, _LOSS, _DATASET_NAME, _LABLE_TYPE, _note, trainingset_to_string(trainingset))
_LOG_DIR = '..\\{}\\{}'.format(output_path[storage], _NET_DIR)

if(not os.path.exists(_LOG_DIR)):
    os.makedirs(_LOG_DIR)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(_LOG_DIR))

device_file = open(_LOG_DIR + '{}.txt'.format(_DEVICE), "a+")

../../data_generation/dataset/201019_2253_final/rgb_proto/lables_ks_proto.csv
Directory >>| ..\fast_output\SYNTH_Regression_mean_squared_error\201019_2253_final_Proto_Top_1_Custom-MAE\Synth_TD\ |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)


### Test to see if csv and images can be found

In [1]:
print(_IMAGE_COUNT)

from IPython.display import Image
print(_CSV_FILE)
df2 = pd.read_csv(_CSV_FILE)
print(df2.head(5))
_IMAGE_NAME = df2.iloc[0]['Filename']
#Image( "../../data_generation/dataset/201019_2253_final/rgb_proto/1-bunny-0-1-290-35.png")
Image(_IMAGE_DIR + _IMAGE_NAME)

NameError: name '_IMAGE_COUNT' is not defined

### Generierung Datenpipeline 

In [69]:
def create_data():
    # if Block für synthetische Daten, um nur auf realen Daten zu trainieren _USE_SYNTHETIC_TRAIN_DATA
    # 1. lege df_train und df_valid als leere Liste an
    # 2. If-block um Zeile df = ... bis df_valid
    
    if trainingset == TrainingSet.SYNTHETIC:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(_SAMPLES * 0.8)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(_SAMPLES * 0.2)]
        
    elif trainingset == TrainingSet.MIXED:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_train = df_shuffled[0 : int(_SAMPLES * 0.8 // _BATCH_SIZE * _BATCH_SIZE)]
        df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(_SAMPLES * 0.2 // _BATCH_SIZE * _BATCH_SIZE)]
        
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train_real = df_shuffled_real[0: int(df_shuffled_real.shape[0] * 0.8 // _BATCH_SIZE * _BATCH_SIZE)]   
        df_valid_real = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train_real.shape[0]])
        df_train = df_train.drop(df_train.index[df_train.shape[0] - df_train_real.shape[0] : df_train.shape[0]])
        df_valid = df_valid.drop(df_valid.index[df_valid.shape[0] - df_valid_real.shape[0] : df_valid.shape[0]])
        df_train = df_train.append(df_train_real)
        df_valid= df_valid.append(df_valid_real)
    
    elif trainingset == TrainingSet.REAL: # Add check for _SAMPLES, once the real dataset increases
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_shuffled_real = df_shuffled_real.drop(df_shuffled_real.index[(df_shuffled_real.shape[0] - 61) : df_shuffled_real.shape[0]])
        df_train = df_shuffled_real[0 : int(df_shuffled_real.shape[0] * 0.8 // _BATCH_SIZE * _BATCH_SIZE)]   
        df_valid = df_shuffled_real.drop(df_shuffled_real.index[0 : df_train.shape[0]])
        
    else:
        print('Create_Data :: should not have reached here')
       

    if _USE_DATA_AUGMENTATION:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255, 
            width_shift_range = 0.1,
            height_shift_range = 0.1, 
            zoom_range = 0.1,
            brightness_range = (0.25, 0.75),
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
    
    print('Y-Col: {}'.format(get_Label_Names(_LABLE_TYPE)))
    print('Train Data Generator: ', end = '')

    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(_LABLE_TYPE),
        class_mode = 'raw',
        target_size = (_IMG_HEIGHT, _IMG_HEIGHT),
        color_mode = _COLOR_MODE,
        shuffle = True,
        seed = 1,
        batch_size = _BATCH_SIZE
    )
                
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    print('Validation Data Generator: ', end = '')

    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(_LABLE_TYPE),
        class_mode = 'raw',
        target_size = (_IMG_HEIGHT,_IMG_HEIGHT),
        color_mode = _COLOR_MODE,
        shuffle = False,
        seed = 1,
        batch_size = _BATCH_SIZE
    )
                                                        
    return train_generator, valid_generator

### Generierung Modell 

In [95]:
def model_run():#x, y, x_val, y_val):
    K.clear_session()
    
    train_generator, valid_generator = create_data()
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
    dropout_rate = _DROP_OUT
    first_neuron = _FIRST_NEURON
    
    if _ACTIVATION == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = params['leaky_alpha'])
    elif _ACTIVATION == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = alexnet(input_shape=(_IMG_HEIGHT,_IMG_HEIGHT,_COLOR_CHANNELS))
      
    print('_________________________________________________________________')
    print('{:>16} {:>16}'.format('Network Layer', 'Trainable'))
    print('=================================================================')
    for layer in cnn.layers:
        print('{:>16} {:>16}'.format(layer.name, layer.trainable))
    print('_________________________________________________________________\n')
    
    model.add(cnn)
    
    #fc = Sequential()
    #fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    #
    #fc.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    #fc.add(activation_layer)
    #if dropout_rate > 0.0:
    #    fc.add(Dropout(rate = dropout_rate))
    #
    #print('Number Hidden Layers {}'.format(_HIDDEN_LAYERS))
    #hidden_neuron_fraction = first_neuron
    #for i in range(_HIDDEN_LAYERS):
    #    hidden_neuron_fraction = hidden_neuron_fraction // 2
    #    fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
    #    fc.add(activation_layer)
    #    if dropout_rate > 0.0:
    #        fc.add(Dropout(rate = dropout_rate))
    #
    #fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    #model.add(fc)
    
    print('Fully Connected Layers added to Base Network')
    
    print('Using Loss: {} \nand Reduction Metric: {}'.format(
        _LOSS, 
        get_Reduction_Metric(_REDUCTION_METRIC)))
    
    model.compile(
        #optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])*1e-2),
        optimizer = _OPTIMIZER(lr = lr_normalizer(_LR, _OPTIMIZER) * 1e-3),
        loss = _LOSS,
        metrics = get_Reduction_Metric(_REDUCTION_METRIC)
    )
    print('Model was compiled')
    print(model.summary())
    print('_________________________________________________________________')
    
    checkpointer = callbacks.ModelCheckpoint(
        #filepath = _LOG_DIR + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        filepath = _LOG_DIR + 'CNN_Base_Model_and_Weights_AlexNet.hdf5',
        monitor =  _MONITOR_VALUE,
        verbose = 1,
        save_weights_only = False,
        save_best_only = True,
        mode = 'min'
    )
    print('Checkpointer was created')
    
    csv_logger = callbacks.CSVLogger(
        #filename = _LOG_DIR + 'CNN_Base_{}_Logger_{}.csv'.format(_MODEL_TO_LOAD_INDEX, train_generator.n),
        filename = _LOG_DIR + 'CNN_Base_Logger_AlexNet.csv',
        separator = ',',
        append = False
    )
    print('CSV Logger was created')

    lr_reducer = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss',
        factor = 0.1,
        patience = 13,
        verbose = 1,
        mode = 'min',
        min_delta = 0.0001
    )
    print('Learning Rate Reducer was created')
    
    early_stopper = callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0,
        #patience = 15,
        patience = 20,
        verbose = 1,
        mode = 'min',
        restore_best_weights = True
    )
    print('Early Stopper was created')
    
    out = model.fit(
        x = train_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_data = valid_generator,
        validation_steps = vg_validation_steps,
        callbacks = [checkpointer, csv_logger, lr_reducer, early_stopper],
        epochs = _EPOCHS,
        workers = 8
    )
    
    return out, model

In [96]:
out, model = model_run()

out.save('AlexNet_Model/')

Y-Col: ['Elevation', 'Azimuth']
Train Data Generator: Found 51 validated image filenames.
Validation Data Generator: Found 12 validated image filenames.
Steps per Epoch: 1, Validation Steps: 0
_________________________________________________________________
   Network Layer        Trainable
        conv2d_1                1
batch_normalization_1                1
 max_pooling2d_1                1
        conv2d_2                1
batch_normalization_2                1
 max_pooling2d_2                1
        conv2d_3                1
batch_normalization_3                1
        conv2d_4                1
batch_normalization_4                1
        conv2d_5                1
batch_normalization_5                1
 max_pooling2d_3                1
       flatten_1                1
         dense_1                1
       dropout_1                1
         dense_2                1
       dropout_2                1
         dense_3                1
____________________________________

NameError: name 'lr_normalizer' is not defined